In [1]:
! pip install cma

     |████████████████████████████████| 230 kB 214 kB/s eta 0:00:01


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import timeit

import numpy as np
import cma
from IPython.display import display, clear_output

from anguilla.optimizers.cma import CMA, StrategyParameters, StoppingConditions
from anguilla.fitness.benchmark import Sphere, SumSquares

In [4]:
def test_with_pycma(n=4):
    fitness_function = Sphere()
    initial_point = fitness_function.propose_initial_point(n)
    initial_sigma = 0.3
    cma.fmin(fitness_function, initial_point, initial_sigma, options={'CMA_Active': 0})

test_with_pycma()

(4_w,8)-CMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=922544, Tue Dec  1 14:41:17 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      8 2.650231767300454e+00 1.0e+00 3.18e-01  3e-01  4e-01 0:00.0
    2     16 2.011003680851720e+00 1.3e+00 3.54e-01  3e-01  4e-01 0:00.0
    3     24 1.421651846753808e+00 1.4e+00 3.57e-01  3e-01  4e-01 0:00.0
  100    800 1.755719619789397e-12 1.6e+00 2.47e-05  7e-07  9e-07 0:00.3
  114    912 1.541075555804607e-14 1.3e+00 5.65e-06  1e-07  1e-07 0:00.3
termination on tolfun=1e-11 (Tue Dec  1 14:41:21 2020)
final/bestever f-value = 4.586217e-15 4.586217e-15
incumbent solution: [2.299713954011094e-08, -8.504234102919866e-09, -1.1290893213914133e-08, 6.21091130009141e-08]
std deviation: [1.0449667141321544e-07, 1.1329506118539404e-07, 1.1141398945644381e-07, 1.0260906509996936e-07]


In [7]:
def test_with_prototype(n=4):
    parameters = StrategyParameters(n=n)
    conditions = StoppingConditions(n=n,
                                    population_size=parameters.population_size,
                                    ftarget=1e-14,
                                    )
    fitness_function = Sphere()
    initial_point = fitness_function.propose_initial_point(n)
    optimizer = CMA(initial_point=initial_point,
                    initial_sigma=0.3,
                    strategy_parameters=parameters,
                    stopping_conditions=conditions)
    while not optimizer.stop():
        candidate_solutions = optimizer.ask()
        function_values = fitness_function(candidate_solutions.T)
        function_evaluations = fitness_function.eval_count
        ranked_indices = optimizer.rank(candidate_solutions, function_values, function_evaluations)
        optimizer.tell(candidate_solutions, ranked_indices)
        if (optimizer._fevals % 20) == 0:
            display(optimizer._fevals, optimizer._best_value, optimizer._best_solution)
    display(optimizer._fevals, optimizer._best_value, optimizer._best_solution)

test_with_prototype(n=4)

20

0.6529910208337285

array([ 0.4787809 ,  0.51181387, -0.35753674,  0.18432015])

40

0.15357226629061202

...

3.631980714436554e-15

array([-4.81402844e-08,  1.46505121e-08, -3.03936228e-08,  1.32696614e-08])

820

3.631980714436554e-15

array([-4.81402844e-08,  1.46505121e-08, -3.03936228e-08,  1.32696614e-08])